<a href="https://colab.research.google.com/github/moyaa05/PracticasAprendizajeAutomatico/blob/PR4/setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Celda 1: Configuración e Importación
Cargamos las librerías necesarias y conectamos con Google Drive. Aquí definimos dónde se guardará todo.

In [1]:
# Importación de librerías
import pandas as pd
import os
import shutil
import numpy as np
from google.colab import drive

# 1. MONTAR GOOGLE DRIVE
print("--- Conectando con Google Drive ---")
drive.mount('/content/drive')

# Configuración de rutas
DRIVE_FOLDER = '/content/drive/MyDrive/Practica_ISIC'
RAW_DATA_DIR = 'ISIC_2019_Training_Input' # Nombre de la carpeta al descomprimir
BASE_DIR = 'dataset' # Nombre de la carpeta organizada para YOLO
SEED = 42 # Semilla típica que ponemos siempre como random

print(f"Directorio de destino configurado: {DRIVE_FOLDER}")

--- Conectando con Google Drive ---
Mounted at /content/drive
Directorio de destino configurado: /content/drive/MyDrive/Practica_ISIC


# Celda 2: Descarga y Descompresión de Datos
Descargamos los datos crudos del servidor del desafío ISIC.

In [2]:
# 2. DESCARGA DE DATOS
print("--- Descargando Dataset ISIC 2019 ---")
# Descargamos el ZIP de imágenes y el CSV de etiquetas
!wget -q "https://isic-challenge-data.s3.amazonaws.com/2019/ISIC_2019_Training_Input.zip"
!wget -q "https://isic-challenge-data.s3.amazonaws.com/2019/ISIC_2019_Training_GroundTruth.csv"

# Verificamos que se bajaron
if os.path.exists("ISIC_2019_Training_Input.zip"):
    print("Descarga completada. Descomprimiendo imágenes... (esto tardará un poco)")
    !unzip -q ISIC_2019_Training_Input.zip
    print("¡Descompresión finalizada!")
else:
    print("Error en la descarga.")

--- Descargando Dataset ISIC 2019 ---
Descarga completada. Descomprimiendo imágenes... (esto tardará un poco)
¡Descompresión finalizada!


# Celda 3: Procesamiento del CSV (Etiquetas)
Aquí transformamos la matriz del formato con el que venía (one-hot encoding) a una lista de categorías legible.

In [3]:
# 3. PROCESAMIENTO DEL DATAFRAME
print("--- Procesando etiquetas ---")
df = pd.read_csv('ISIC_2019_Training_GroundTruth.csv')

# Convertimos One-Hot a Categoría única usando idxmax
# Excluimos la columna 'image' para buscar el valor máximo (1.0) en las columnas de enfermedades
df['clase'] = df.drop('image', axis=1).idxmax(axis=1)

# Añadimos la extensión .jpg a los nombres de imagen para facilitar la copia posterior
df['image_name'] = df['image'] + '.jpg'

# Mostramos un resumen para verificar
print("Clases detectadas y cantidad de imágenes originales:")
display(df['clase'].value_counts())
print("\nPrimeras filas del dataframe procesado:")
display(df[['image', 'clase']].head())

--- Procesando etiquetas ---
Clases detectadas y cantidad de imágenes originales:


,count
clase,
NV,12875
MEL,4522
BCC,3323
BKL,2624
AK,867
SCC,628
VASC,253
DF,239



Primeras filas del dataframe procesado:


,image,clase
0,ISIC_0000000,NV
1,ISIC_0000001,NV
2,ISIC_0000002,MEL
3,ISIC_0000003,NV
4,ISIC_0000004,MEL


# Celda 4: Muestreo Estratificado y Estructura de Carpetas
Seleccionamos 100 imágenes para entrenar y 10 para test, y creamos las carpetas físicas.

In [4]:
# 4. CREACIÓN DE ESTRUCTURA Y COPIA DE ARCHIVOS
print("--- Generando Dataset Balanceado ---")

# Limpiamos si existía de una ejecución anterior para evitar mezclas
if os.path.exists(BASE_DIR):
    shutil.rmtree(BASE_DIR)

# Iteramos por cada enfermedad (clase)
for clase in df['clase'].unique():
    print(f"Procesando clase: {clase}...")

    # 1. Crear carpetas físicas (dataset/train/MEL, dataset/test/MEL, etc.)
    os.makedirs(os.path.join(BASE_DIR, 'train', clase), exist_ok=True)
    os.makedirs(os.path.join(BASE_DIR, 'test', clase), exist_ok=True)

    # 2. Filtrar y Muestrear
    df_clase = df[df['clase'] == clase]

    # Seleccionamos 110 imágenes aleatorias
    if len(df_clase) >= 110:
        muestras = df_clase.sample(n=110, random_state=SEED)
        train_imgs = muestras.iloc[:100]  # Primeras 100
        test_imgs = muestras.iloc[100:110] # Últimas 10
    else:
        # Por si alguna clase tuviera menos (no debería pasar en este dataset, pero supuestamente es buena práctica)
        print(f"⚠️ Advertencia: {clase} tiene menos de 110 imágenes. Se usarán todas.")
        muestras = df_clase
        # Lógica simple de división 90/10 si faltan datos
        split_idx = int(len(muestras) * 0.9)
        train_imgs = muestras.iloc[:split_idx]
        test_imgs = muestras.iloc[split_idx:]

    # 3. Copiar archivos físicos
    # Copiamos a TRAIN
    for _, row in train_imgs.iterrows():
        src = os.path.join(RAW_DATA_DIR, row['image_name'])
        dst = os.path.join(BASE_DIR, 'train', clase, row['image_name'])
        try:
            shutil.copy(src, dst)
        except FileNotFoundError:
            print(f"Falta archivo: {row['image_name']}")

    # Copiamos a TEST
    for _, row in test_imgs.iterrows():
        src = os.path.join(RAW_DATA_DIR, row['image_name'])
        dst = os.path.join(BASE_DIR, 'test', clase, row['image_name'])
        try:
            shutil.copy(src, dst)
        except FileNotFoundError:
            print(f"Falta archivo: {row['image_name']}")

print("\n¡Organización completada!")

--- Generando Dataset Balanceado ---
Procesando clase: NV...
Procesando clase: MEL...
Procesando clase: BKL...
Procesando clase: DF...
Procesando clase: SCC...
Procesando clase: BCC...
Procesando clase: VASC...
Procesando clase: AK...

¡Organización completada!


# Celda 5: Verificación de Resultados
Es muy importante verificar que las carpetas tienen lo que esperamos antes de guardar.

In [5]:
# 5. VERIFICACIÓN
# Contamos cuántos archivos hay en cada carpeta para asegurar que tenemos 100 y 10.
print("--- Verificación del Dataset Generado ---")

for split in ['train', 'test']:
    print(f"\nConjunto: {split.upper()}")
    split_path = os.path.join(BASE_DIR, split)
    for clase in os.listdir(split_path):
        clase_path = os.path.join(split_path, clase)
        count = len(os.listdir(clase_path))
        print(f"  - {clase}: {count} imágenes")

--- Verificación del Dataset Generado ---

Conjunto: TRAIN
  - VASC: 100 imágenes
  - BKL: 100 imágenes
  - SCC: 100 imágenes
  - MEL: 100 imágenes
  - DF: 100 imágenes
  - NV: 100 imágenes
  - AK: 100 imágenes
  - BCC: 100 imágenes

Conjunto: TEST
  - VASC: 10 imágenes
  - BKL: 10 imágenes
  - SCC: 10 imágenes
  - MEL: 10 imágenes
  - DF: 10 imágenes
  - NV: 10 imágenes
  - AK: 10 imágenes
  - BCC: 10 imágenes


# Celda 6: Empaquetado y Guardado en Drive
Finalmente, creamos el ZIP y lo enviamos al "almacén".

In [6]:
# 6. GUARDAR EN DRIVE
print("--- Empaquetando y Guardando ---")

# Nombre del archivo zip
zip_name = 'dataset_ISIC.zip'

# Comprimir la carpeta 'dataset'
print(f"Comprimiendo carpeta '{BASE_DIR}'...")
!zip -r -q {zip_name} {BASE_DIR}

# Mover a Drive
dest_path = os.path.join(DRIVE_FOLDER, zip_name)
print(f"Copiando a {dest_path}...")
shutil.copy(zip_name, dest_path)

print(f"\n✅ ¡ÉXITO! Archivo guardado correctamente en: {dest_path}")

--- Empaquetando y Guardando ---
Comprimiendo carpeta 'dataset'...
Copiando a /content/drive/MyDrive/Practica_ISIC/dataset_ISIC.zip...

✅ ¡ÉXITO! Archivo guardado correctamente en: /content/drive/MyDrive/Practica_ISIC/dataset_ISIC.zip
